# Automatically retrieve information from Indexfungorum using Selenium.

**Index Fungorum** is a dynamic webpage, making it impossible to directly scrape certain information using Python. [Link to Index Fungorum](https://indexfungorum.org/Names/Names.asp).


**Selenium** provides an interactive way for humans to interface with Index Fungorum through user input.

In [69]:
# Load required packages
import re
import os
import sys
import time
import pandas as pd
import requests

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from Bio import Entrez

from tqdm import tqdm # Providing progress bar,to monitor the program’s progress if the group contains thousands of records.

# Define functions

In [70]:
def parse_search_page(html_doc):
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    RESET = '\033[0m'  # Reset color to default
    
    # Regex pattern to extract content between 'BofP</a>' and '<br><p><b>Pages:'
    pattern = r'BofP</a>(.*?)<br><p><b>Pages:'
    matches = re.findall(pattern, html_doc, re.DOTALL)
    
    if len(matches) == 1:
        record_lst = matches[0].split("<br>")
    else:
        sys.exit(f"Failed to extract the expected content. Check function: parse_search_page")

    # List to hold parsed record information
    records = []

    # Iterate through the extracted record lines
    for line in record_lst:
        # Regex pattern to extract record name and link
        try:
            record_name_link_pattern = r'href="(NamesRecord.asp\?RecordID=\d+)">([^<]+)'  # Match the record name and link
            match0 = re.search(record_name_link_pattern, line, re.DOTALL)
            record_name = match0.group(2)  # Get the record name
            record_link = "https://indexfungorum.org/Names/" + match0.group(1)  # Construct the full URL
        except:
            print(f"{RED}[Warning]{RESET} | {GREEN}{line}{RESET}")
            record_name = "NA"
            record_link = "NA"

        # Regex pattern to extract author and year
        try:
            author_year_pattern = r'</a>(.*?) (\d{4})'  # Match author and year after closing </a>
            match1 = re.search(author_year_pattern, line, re.DOTALL)
            record_author = match1.group(1).replace("&amp;","&").strip()  # Get the author
            year = match1.group(2)  # Get the year
            #print(record_author,"|",year)
        except:
            print(f"{RED}[Warning]{RESET} | {GREEN}{line}{RESET}")
            record_author = "NA"
            year = "NA"

        # Regex pattern to extract current name and link
        try:
            current_name_link_pattern = r'(http://www.speciesfungorum.org/Names/SynSpecies.asp\?RecordID=\d+)">([^<]+)'  # Match current name and link
            match2 = re.search(current_name_link_pattern, line)
            current_name = match2.group(2)  # Get the current name
            current_name_link = match2.group(1)  # Get the current name link
            #print(current_name,"|",current_name_link)
        except:
            print(f"{RED}[Warning]{RESET} | {GREEN}{line}{RESET}")
            current_name = "NA"
            current_name_link = "NA"

        # Append the record information as a list
        records.append([record_name, record_author, year, record_link, current_name, current_name_link])

    return records

In [71]:
def search(group_name):
    '''Input a record name and search for it.'''
    
    # Locate and clear the search box, then enter the search term
    search_box = driver.find_element(By.NAME, 'SearchTerm')
    search_box.clear()
    search_box.send_keys(group_name)  # Use the correct variable (group_name)
    
    # Click the search button to initiate the search
    search_btn = driver.find_element(By.CSS_SELECTOR, '[type="submit"]')
    search_btn.click()
    
    # Wait for the page to load
    # time.sleep(5)

    try:
        # Extract the number of records displayed on the page
        number_of_records = driver.find_element(By.XPATH, '/html/body/table/tbody/tr[2]/td/table/tbody/tr/td/p[1]/b[3]').text.split()[1]
        number_of_records = int(number_of_records)
    except Exception as e:
        print(f"Error: Failed to retrieve the number of records. {e}")
        return []

    # Calculate the number of pages required to display all records
    num_page = (number_of_records + 199) // 200  # This simplifies the page calculation logic
    
    # Output the number of pages and records for debugging
    print(f'Number of pages   : {num_page}', file=sys.stdout, flush=True)
    print(f'Number of records : {number_of_records}', file=sys.stdout, flush=True)
    
    #time.sleep(2)

    # Initialize a list to store all records
    all_records_lst = []

    # Parse the first page
    html = driver.page_source
    records_lst = parse_search_page(html)
    all_records_lst.extend(records_lst)
    
    # Parse additional pages, if any
    if num_page >= 2:
        for i in range(2, num_page + 1):
            try:
                # Locate and click the pagination button for the next page
                css_pattern = f'[href="Names.asp?pg={i}"]'
                page_btn = driver.find_element(By.CSS_SELECTOR, css_pattern)
                page_btn.click()

                # Wait for the next page to load
                #time.sleep(5)

                # Parse the page source for records
                html = driver.page_source
                records_lst = parse_search_page(html)
                all_records_lst.extend(records_lst)
            except:
                sys.exit(1)

    # Check if the number of records matches the expected count
    if len(all_records_lst) != number_of_records:
        print('Warning: Some records did not contain links.')
    
    # Return the list of all records
    return all_records_lst

In [72]:
def check_nucleotide_records(df):
    Entrez.email = "chenyanpeng1992@outlook.com"
    unique_term_nucleotide_count_dict = {}
    current_name_lst = list(set(df["Current_name"]))
    
    for search_nucleotide_term in tqdm(current_name_lst,desc="query NCBI nucleotide"):
        search_nucleotide_term = " ".join(search_nucleotide_term.split()[0:2])
        if search_nucleotide_term not in unique_term_nucleotide_count_dict:
            handle = Entrez.esearch(db="nucleotide", term=search_nucleotide_term)
            records = Entrez.read(handle)
            unique_term_nucleotide_count_dict[search_nucleotide_term] = records['Count']
    df['Nt_count'] = df['Current_name'].map(unique_term_nucleotide_count_dict)
    return df

In [73]:
def output(search_term, df):
    # Export the DataFrame to Excel
    time_suffix = time.strftime('%Y%m%d')
    output_file = search_term + '_indexfungorum_records_' + str(time_suffix) + '.xlsx'
    df.to_excel(output_file, index=False)

# Use

## Launch Indexfungorum

In [87]:
s = Service(r'msedgedriver.exe')
option = webdriver.EdgeOptions()

#option.add_argument('--headless')  # hide search window
#driver = webdriver.Edge(service=s, options = option)
driver = webdriver.Edge(service=s)

#driver.implicitly_wait(5) # active Edge searching engine and 5s for the browser to load the dat
driver.get('http://www.indexfungorum.org/')

search_index_fungorum_element = driver.find_element(By.CSS_SELECTOR, '[href="./Names/Names.asp"]')
search_index_fungorum_element.click()
search_box = driver.find_element(By.NAME, 'SearchTerm')
search_box.clear()

## Search and parse

In [88]:
search_term = 'Uromyces'
records_lst = search(search_term)
driver.close()

Number of pages   : 8
Number of records : 1585
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=428152">Uromyces acantholimonis var. acantholimonis</a> Syd. &amp; P. Syd. 1906; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=621952">Uromyces acantholimonis var. zagrosicus</a> Z. Urb. 1967; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=452529">Uromyces acuminatus var. acuminatus</a> Arthur 1883; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=454189">Uromyces acuminatus var. spartinae</a> (Farl.) Arthur 1883; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=140944">Uromyces adenostylis</a> E. Fisch. 1908; <a href="families.asp?FamilyName=Pucciniac

[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=176983">Uromyces borealis</a> Liro 1908; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=150665">Uromyces brassicae</a> Niessl 1872; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=586120">Uromyces bulbosi-dactylidis</a> Kleb. 1907; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=586619">Uromyces burrillii</a> Lagerh. 1895; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=515777">Uromyces caricinus</a> (DC.) Lév. 1847; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=163874">Uromyces caricinus</a> Ellis &amp; Everh. 1895

[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=266412">Uromyces graminis f.sp. foeniculi-melicae</a> A.L. Guyot 1938; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=266413">Uromyces graminis f.sp. melicae-ferulae</a> Maire 1902; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=625975">Uromyces graminis f.sp. punicus</a> Maire 1902; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=420270">Uromyces handelii var. handelii</a> Bubák 1914; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=137986">Uromyces handelii var. meliloti</a> Trotter 1942; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="Name

[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=138775">Uromyces ornithogali subsp. gageae</a> Beck 1885; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=420964">Uromyces ornithogali subsp. ornithogali</a> (J.C. Schmidt &amp; Kunze) Niessl 1864; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=273077">Uromyces orobi f.sp. viciae-nipponicae</a> Hirats. f. 1933; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=506426">Uromyces orobi f.sp. viciae-unijugae</a> Hirats. f. 1933; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=544148">Uromyces oxytropis</a> J. Kunze 1874; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class

[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=515766">Uromyces rhinanthacearum</a> (DC.) Lév. 1847; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=202959">Uromyces rhynchosiae</a> Cooke ex Sacc. 1888; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=307580">Uromyces rhynchosiae</a> Cooke ex Doidge 1948; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=202998">Uromyces rhynchosporicola</a> Henn. 1896; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=276686">Uromyces romuleae</a> Doidge 1927; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=277119">Uromyces rumicis f.s

[Warning] | </p><a class="LinkColour1" href="NamesRecord.asp?RecordID=174656">Uromyces suaedae</a> Jacz. 1893; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=344315">Uromyces suffruticosae</a> Vienn.-Bourg. 1954; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=307590">Uromyces suzukii</a> Sawada 1943; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=174456">Uromyces symphiostemi</a> Speg. 1887; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=155551">Uromyces tanaceti</a> Rabenh. 1844; <a href="families.asp?FamilyName=Pucciniaceae">Pucciniaceae</a>
[Warning] | <a class="LinkColour1" href="NamesRecord.asp?RecordID=155278">Uromyces taubertii</a> Henn. 1892; <a href="

In [89]:
df = pd.DataFrame(records_lst, columns=['Name', 'Author', 'Year', 'Name_link', 'Current_name', 'Synonym_link'])
df['Current_name'] = df.apply(
    lambda row: " ".join(row['Name'].split()[0:2]) if row['Current_name'] == "NA" else row['Current_name'],
    axis=1
)

## Check if the name has sequence data

In [90]:
df = check_nucleotide_records(df)

query NCBI nucleotide: 100%|███████████████████████████████████████████████████████████████████| 1255/1255 [19:52<00:00,  1.05it/s]


## Output table

In [78]:
output(search_term, df)

# Describe table

In [82]:
count = len(df[df['Current_name'].apply(lambda x: x.split()[0] != search_term)])
print(f"{count} name records were revised.")

2 name records were revised.


In [86]:
current_count = len(df[df['Current_name'].apply(lambda x: x.split()[0] == search_term)])
print(f"{current_count} name records are valid right now.")
df[df['Current_name'].apply(lambda x: x.split()[0] == search_term)]["Current_name"].tolist()

11 name records are valid right now.


['Neottiosporina apoda',
 'Neottiosporina ashworthiae',
 'Neottiosporina asymmetrica',
 'Neottiosporina australiensis',
 'Neottiosporina clavata',
 'Neottiosporina cylindrica',
 'Neottiosporina masonii',
 'Neottiosporina mihintaleensis',
 'Neottiosporina paspali',
 'Neottiosporina phragmiticola',
 'Neottiosporina sorghicola']